In [ ]:
# os.environ["MODIN_ENGINE"] = "dask"
# import modin.pandas as pd
import pandas as pd
import numpy as np
import requests
import math
import xlsxwriter
from scipy import stats
from credentials import IEX_CLOUD_API_TOKEN

In [ ]:
stocks = pd.read_csv('sp-500-stocks.csv')

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

In [ ]:
data['year1ChangePercent']

In [ ]:
columns = ["Tickers", "Price", "One-Year Price Return", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=columns)

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

In [ ]:
for symbol_string in symbol_strings:
    batch_api_call_url = \
        f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} \
            &types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]["price"], 
                    data[symbol]["stats"]["year1ChangePercent"], 
                    "N/A"
                ], 
                index=columns
            ), 
            ignore_index=True
        )
final_dataframe

In [ ]:
final_dataframe.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
final_dataframe[:50]